In [0]:
#Required ecah time the cluster is restarted wich should be only on the first notebook as they run in order

tiers = ['bronze','silver','gold']
adls_path = {tier: f'abfss://{tier}@sadbfull.dfs.core.windows.net/' for tier in tiers}

# Acessing paths
bronze_adls = adls_path['bronze']
silver_adls = adls_path['silver']
gold_adls =   adls_path['gold']

dbutils.fs.ls(bronze_adls)
dbutils.fs.ls(silver_adls)
dbutils.fs.ls(gold_adls)

[FileInfo(path='abfss://gold@sadbfull.dfs.core.windows.net/earthquake_events_gold/', name='earthquake_events_gold/', size=0, modificationTime=1739214909000)]

In [0]:
import requests
import json
from datetime import date, timedelta


In [0]:
start_date = date.today() - timedelta(1)
end_date = date.today()

print(start_date)
print(end_date)

2025-02-09
2025-02-10


In [0]:
%python
import requests
import json

# Construct the API URL with start and end dates provided by DataFactory, formatted by geojson output
url = f"https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime={start_date}&endtime={end_date}"

try:
    # Make the GET request to fetch data
    response = requests.get(url)

    # Check if the request was successful
    response.raise_for_status()  # Raise HTTPError for bad responses (4xx 5xx)
    data = response.json().get('features', [])

    if not data:
        print('No data returned for the specified date range')
    else:
        # Specify the ADLS path
        file_path = f"{bronze_adls}/{start_date}_earthquake.json"

        # Save the JSON data
        json_data = json.dumps(data, indent=4)
        dbutils.fs.put(file_path, json_data, overwrite=True)
        print(f"Data successfully saved to {file_path}")

except requests.exceptions.RequestException as e:
    print(f"Error fetching data from API: {e}")

Wrote 307034 bytes.
Data successfully saved to abfss://bronze@sadbfull.dfs.core.windows.net//2025-02-09_earthquake.json


In [0]:
#Define your variables
output_data = {
    "start_date": start_date.isoformat(),
    "end_date": end_date.isoformat(),
    "bronze_adls": bronze_adls,
    "silver_adls": silver_adls,
    "gold_adls": gold_adls
}

#Return the dictionary directly
dbutils.jobs.taskValues.set(key = 'bronze_output', value= output_data)